In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from FeatureEngine.FeatureMethods import FeatureEngine
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import numpy as np
from pathlib import Path

#Note: The FeatureEngine is for TESTING ONLY, we'll need to pickle sklearn objects which means the methods as they are currentlly set-up in function featurizer won't handle pickling.
#https://stackoverflow.com/questions/28656736/using-scikits-labelencoder-correctly-across-multiple-programs


print(Path.cwd())

pathBase = str(Path.cwd())
cv = CountVectorizer()


#df1 = pd.read_csv(pathBase + "\Datasets\dataset.csv")

df = pd.read_csv(pathBase + "\Datasets\TeraSample.csv")


#categories = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med']

#twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)
#venv guidance
#https://stackoverflow.com/questions/8921188/issue-with-virtualenv-cannot-activate
#https://janakiev.com/blog/jupyter-virtual-envs/
#https://towardsdatascience.com/the-art-of-effective-visualization-of-multi-dimensional-data-6c7202990c57

c:\Users\dylan.levine\Development\HCP Classification\HCP-Classification


In [12]:
#Select columns, replace "?" with numpy nan type and initialize FeatureEngine object

colSelect = ['COMP_CODE','AC_DOC_NR','PSTNG_DATE', 'CREATEDON','DOC_DATE', 'PROFIT_CTR', 'AC_DOC_TYP', 'POST_KEY', 'CREDITOR', 'NAME', 'DEB_CRE_DC', 'GL_ACCOUNT', 'CUSTOMER', 'POSTXT', 'BIC_ZUSERNAM', 'REF_KEY1', 'ZBKTXT', 'ZTCODE', 'DEB_CRE_USD', 'PROD_DESCR', 'BIC_ZTERMPAY', 'MATL_GROUP', 'ACCNT_GRPV']

fieldType = ['Categorical', 'ID', 'Date', 'Date', 'Date', 'Categorical', 'Categorical', 'Categorical', 'Categorical', 'Text', 'Numerical', 'Categorical', 'Categorical', 'Text', 'Categorical', 'Categorical', 'Text', 'Categorical', 'Numerical', 'Categorical', 'Categorical', 'Categorical', 'Categorical']

#REF_KEY1 fill "?" with nan
#Do we have a product # vs product description?
#Classify BIC_ZUSERNAM as manual or not depending on if posted by AUTOSYS
#Tokenize appended text fields of NAME, POSTXT, ZBTXT
#Feature ideas, calculate diff of posting and created?
#n_samples > n_features ** 2 (i.e. rows > columns.count squared)
df = df[colSelect].replace("?", np.nan)
df['REF_KEY1'] = df['REF_KEY1'].replace("?", "nan_REF_KEY1")

df['textappend'] = df['NAME'].astype(str) +" "+ df['POSTXT'].astype(str) +" "+ df["ZBKTXT"].astype(str)

modelVals = FeatureEngine(df)
df.head()


,COMP_CODE,AC_DOC_NR,PSTNG_DATE,CREATEDON,DOC_DATE,PROFIT_CTR,AC_DOC_TYP,POST_KEY,CREDITOR,NAME,...,BIC_ZUSERNAM,REF_KEY1,ZBKTXT,ZTCODE,DEB_CRE_USD,PROD_DESCR,BIC_ZTERMPAY,MATL_GROUP,ACCNT_GRPV,textappend
0,1003,5000015131,20200124,20200124,20200124,12071,WE,86,10026901,ALERE SAN DIEGO,...,GRIFFLX5,NaN,NaN,MIGO_GR,231.39,OXYCODONE CONTROL SET 2,23,Z1-102126,Z001,ALERE SAN DIEGO 0157628 01/24/20 1:00 nan
1,1003,5000182043,20190930,20190930,20190930,12262,WE,81,10027697,MERGON INTERNATIONAL,...,AUTOSYS,NaN,NaN,MB01,"13,445.19",NaN,24,NaN,Z001,MERGON INTERNATIONAL 2L ALINITY BTLE VALIDATIO...
2,1003,5100010932,20200117,20200117,20200116,12259,RE,81,10003556,"EXPERIS US, INC.",...,AUTOSYS,EDIINV,NaN,MR1M,"1,919.60",NaN,23,NaN,Z001,"EXPERIS US, INC. nan nan"
3,1003,5000180876,20190927,20190927,20190927,12071,WE,96,10032383,BIO RAD LABORATORIES INC,...,BERGESX6,NaN,NaN,MIGO_GR,-513.45,BR148 CRDMKR LT+2,17,Z1-102126,Z001,BIO RAD LABORATORIES INC nan nan
4,1003,5100008364,20190114,20190114,20190110,12071,RE,86,30015883,NYPRO INC,...,RENZAC,MANUAL,NaN,MR1M,"106,963.20",ARC REACTION VESSELS,24,Z1-102130,Z003,NYPRO INC nan nan


In [13]:
modelVals.oneHotEncodeLabels(["COMP_CODE","PROFIT_CTR","AC_DOC_TYP","POST_KEY","GL_ACCOUNT","ACCNT_GRPV","BIC_ZTERMPAY"])


#modelVals.oneHotEncodeLabels("BIC_ZTERMPAY")

modelVals.df

,AC_DOC_NR,PSTNG_DATE,CREATEDON,DOC_DATE,CREDITOR,NAME,DEB_CRE_DC,CUSTOMER,POSTXT,BIC_ZUSERNAM,...,ACCNT_GRPV_Z004,BIC_ZTERMPAY_1,BIC_ZTERMPAY_13,BIC_ZTERMPAY_17,BIC_ZTERMPAY_19,BIC_ZTERMPAY_23,BIC_ZTERMPAY_24,BIC_ZTERMPAY_30,BIC_ZTERMPAY_46,BIC_ZTERMPAY_47
0,5000015131,20200124,20200124,20200124,10026901,ALERE SAN DIEGO,231.39,NaN,0157628 01/24/20 1:00,GRIFFLX5,...,0,0,0,0,0,1,0,0,0,0
1,5000182043,20190930,20190930,20190930,10027697,MERGON INTERNATIONAL,"12,150.00",NaN,2L ALINITY BTLE VALIDATION 50%,AUTOSYS,...,0,0,0,0,0,0,1,0,0,0
2,5100010932,20200117,20200117,20200116,10003556,"EXPERIS US, INC.","1,919.60",NaN,NaN,AUTOSYS,...,0,0,0,0,0,1,0,0,0,0
3,5000180876,20190927,20190927,20190927,10032383,BIO RAD LABORATORIES INC,-513.45,55872485,NaN,BERGESX6,...,0,0,0,1,0,0,0,0,0,0
4,5100008364,20190114,20190114,20190110,30015883,NYPRO INC,"106,963.20",NaN,NaN,RENZAC,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,5100110476,20190605,20190605,20190603,10002390,MAINE STANDARDS COMPANY LLC,6.2,NaN,NaN,C_ERP_BRAINW,...,0,0,0,0,0,0,1,0,0,0
1996,5000077981,20190425,20190425,20190425,10154037,NYPRO CHICAGO,"-9,253.44",NaN,NaN,DIAZJA5,...,0,0,0,0,0,0,1,0,0,0
1997,5000119824,20200625,20200625,20200603,10034350,STERIGENICS US LLC,"-4,225.00",NaN,NaN,ANDINEX1,...,0,0,1,0,0,0,0,0,0,0
1998,5000237416,20191219,20191219,20191219,10032383,BIO RAD LABORATORIES INC,-189,50143536,NaN,BERGESX6,...,0,0,0,1,0,0,0,0,0,0


In [32]:
modelVals = FeatureEngine(df1)

modelVals.df.describe()

,id,HCP
count,1000.000000,1000.00000
mean,500.500000,0.49000
std,288.819436,0.50015
min,1.000000,0.00000
25%,250.750000,0.00000
50%,500.500000,0.00000
75%,750.250000,1.00000
max,1000.000000,1.00000


In [33]:
modelVals = FeatureMethods.FeatureEngine(df1)

modelVals.encodeLabels('EntityName')

modelVals.oneHotEncodeLabels('EntityName')
modelVals.outputFeatures()
modelVals.fit_transform()

modelVals.dfscaled


,EntityName_Acura,EntityName_Alfa Romeo,EntityName_Aston Martin,EntityName_Audi,EntityName_Austin,EntityName_BMW,EntityName_Bentley,EntityName_Buick,EntityName_Cadillac,EntityName_Chevrolet,...,EntityName_Saab,EntityName_Saturn,EntityName_Scion,EntityName_Subaru,EntityName_Suzuki,EntityName_Tesla,EntityName_Toyota,EntityName_Volkswagen,EntityName_Volvo,EntityName_labeled
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.684211
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.315789
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.122807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.649123
996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.964912
997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.298246
998,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.052632


In [6]:
modelVals.df

,id,UserName,TranDescription,HCP,EntityName_labeled,EntityName_Acura,EntityName_Alfa Romeo,EntityName_Aston Martin,EntityName_Audi,EntityName_Austin,...,EntityName_Rolls-Royce,EntityName_Saab,EntityName_Saturn,EntityName_Scion,EntityName_Subaru,EntityName_Suzuki,EntityName_Tesla,EntityName_Toyota,EntityName_Volkswagen,EntityName_Volvo
0,1,wletterese0,Praesent id massa id nisl venenatis lacinia. A...,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,bpenright1,Donec semper sapien a libero. Nam dui. Proin l...,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,amenichelli2,Vestibulum ante ipsum primis in faucibus orci ...,1,39,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,jalessandone3,"Aliquam augue quam, sollicitudin vitae, consec...",0,18,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,gpattrick4,Vestibulum ante ipsum primis in faucibus orci ...,0,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,gmoanrn,Vivamus in felis eu sapien cursus vestibulum. ...,0,37,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,997,jjacombro,In sagittis dui vel nisl. Duis ac nibh. Fusce ...,1,55,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
997,998,wailmerrp,Vestibulum ante ipsum primis in faucibus orci ...,0,17,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,999,mskirvingrq,In congue. Etiam justo.,0,3,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
modelVals.reverse_transform()

modelVals.dfscaled

,EntityName_Acura,EntityName_Alfa Romeo,EntityName_Aston Martin,EntityName_Audi,EntityName_Austin,EntityName_BMW,EntityName_Bentley,EntityName_Buick,EntityName_Cadillac,EntityName_Chevrolet,...,EntityName_Saab,EntityName_Saturn,EntityName_Scion,EntityName_Subaru,EntityName_Suzuki,EntityName_Tesla,EntityName_Toyota,EntityName_Volkswagen,EntityName_Volvo,EntityName_labeled
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.0
996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,55.0
997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0
998,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0


In [4]:


modelVals.concatVector('TranDescription',1,1) 


#.df accesses the now modified dataframe with features added to df
modelVals.df
#modelVals.originalFeatures


,id,UserName,EntityName,TranDescription,HCP,EntityName_labeled,ac,accumsan,adipiscing,aenean,...,vehicula,vel,velit,venenatis,vestibulum,vitae,vivamus,viverra,volutpat,vulputate
0,1,wletterese0,Acura,Praesent id massa id nisl venenatis lacinia. A...,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
1,2,bpenright1,Acura,Donec semper sapien a libero. Nam dui. Proin l...,0,0,2,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,3,amenichelli2,Mercedes-Benz,Vestibulum ante ipsum primis in faucibus orci ...,1,39,0,0,0,0,...,0,0,0,1,1,2,0,1,0,0
3,4,jalessandone3,GMC,"Aliquam augue quam, sollicitudin vitae, consec...",0,18,0,0,0,0,...,0,1,0,0,1,1,0,0,0,0
4,5,gpattrick4,Buick,Vestibulum ante ipsum primis in faucibus orci ...,0,7,1,0,0,0,...,0,1,0,1,2,0,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,gmoanrn,Mazda,Vivamus in felis eu sapien cursus vestibulum. ...,0,37,1,0,1,0,...,0,0,1,0,1,1,1,0,0,0
996,997,jjacombro,Toyota,In sagittis dui vel nisl. Duis ac nibh. Fusce ...,1,55,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
997,998,wailmerrp,Ford,Vestibulum ante ipsum primis in faucibus orci ...,0,17,1,0,0,0,...,0,1,0,1,2,0,0,0,0,0
998,999,mskirvingrq,Audi,In congue. Etiam justo.,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
text_features = cv.fit_transform(df['TranDescription'].values)
vect_df = pd.DataFrame(text_features.todense(), columns=cv.get_feature_names())

df = pd.concat([df, vect_df], axis=1)

df

,id,UserName,EntityName,TranDescription,HCP,ac,accumsan,adipiscing,aenean,aliquam,...,vehicula,vel,velit,venenatis,vestibulum,vitae,vivamus,viverra,volutpat,vulputate
0,1,wletterese0,Acura,Praesent id massa id nisl venenatis lacinia. A...,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
1,2,bpenright1,Acura,Donec semper sapien a libero. Nam dui. Proin l...,0,2,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,3,amenichelli2,Mercedes-Benz,Vestibulum ante ipsum primis in faucibus orci ...,1,0,0,0,0,2,...,0,0,0,1,1,2,0,1,0,0
3,4,jalessandone3,GMC,"Aliquam augue quam, sollicitudin vitae, consec...",0,0,0,0,0,1,...,0,1,0,0,1,1,0,0,0,0
4,5,gpattrick4,Buick,Vestibulum ante ipsum primis in faucibus orci ...,0,1,0,0,0,1,...,0,1,0,1,2,0,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,gmoanrn,Mazda,Vivamus in felis eu sapien cursus vestibulum. ...,0,1,0,1,0,1,...,0,0,1,0,1,1,1,0,0,0
996,997,jjacombro,Toyota,In sagittis dui vel nisl. Duis ac nibh. Fusce ...,1,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
997,998,wailmerrp,Ford,Vestibulum ante ipsum primis in faucibus orci ...,0,1,0,0,0,0,...,0,1,0,1,2,0,0,0,0,0
998,999,mskirvingrq,Audi,In congue. Etiam justo.,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
descriptions = df['TranDescription'].values
y = df['HCP'].values  #y represents the classification value (i.e. HCP vs. not HCP)

descriptions_train, descriptions_test, y_train, y_test = train_test_split(descriptions, y, test_size=0.25, random_state=1000)

descriptions_train.shape
y_train

array([1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1,

In [50]:


#X_train = cv.fit_transform(df['TranDescription'])

descriptions_train_transform = cv.fit_transform(descriptions_train)

vect_df = pd.DataFrame(descriptions_train_transform.todense(), columns=cv.get_feature_names())

vect_df

,ac,accumsan,adipiscing,aenean,aliquam,aliquet,amet,ante,arcu,at,...,vehicula,vel,velit,venenatis,vestibulum,vitae,vivamus,viverra,volutpat,vulputate
0,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,2,0,1,0,0,1
1,0,0,1,1,0,0,2,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,1,0,0,0,0,1,1,...,1,0,0,0,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
746,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
747,0,1,0,0,0,1,2,0,0,0,...,1,2,0,0,1,0,0,0,0,0
748,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,1,0,0,0,0,0


In [45]:
print(cv.get_feature_names())

['ac', 'accumsan', 'adipiscing', 'aenean', 'aliquam', 'aliquet', 'amet', 'ante', 'arcu', 'at', 'auctor', 'augue', 'bibendum', 'blandit', 'commodo', 'condimentum', 'congue', 'consectetuer', 'consequat', 'convallis', 'cras', 'cubilia', 'cum', 'curabitur', 'curae', 'cursus', 'dapibus', 'diam', 'dictumst', 'dignissim', 'dis', 'dolor', 'donec', 'dui', 'duis', 'egestas', 'eget', 'eleifend', 'elementum', 'elit', 'enim', 'erat', 'eros', 'est', 'et', 'etiam', 'eu', 'euismod', 'facilisi', 'faucibus', 'felis', 'fermentum', 'feugiat', 'fringilla', 'fusce', 'gravida', 'habitasse', 'hac', 'hendrerit', 'iaculis', 'id', 'imperdiet', 'in', 'integer', 'interdum', 'ipsum', 'justo', 'lacinia', 'lacus', 'laoreet', 'lectus', 'leo', 'libero', 'ligula', 'lobortis', 'lorem', 'luctus', 'maecenas', 'magna', 'magnis', 'malesuada', 'massa', 'mattis', 'mauris', 'metus', 'mi', 'molestie', 'mollis', 'montes', 'morbi', 'mus', 'nam', 'nascetur', 'natoque', 'nec', 'neque', 'nibh', 'nisi', 'nisl', 'non', 'nonummy', 'null

In [46]:
descriptions_train_transform.toarray()

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)